In [ ]:
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import os

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

In [ ]:
import torchvision.transforms as transforms
trans=transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.GaussianBlur(kernel_size=3, sigma=0.1),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
from sklearn.model_selection import train_test_split
train_data=os.listdir("TRAIN")
all_paths=[]
all_labels=[]
for i,path in enumerate(train_data):
    full_path="TRAIN/"+path+"/"
    lst=os.listdir(full_path)
    lst=[full_path+i for i in lst]
    all_paths.extend(lst)
    all_labels.extend([i for j in range(len(lst))])
all_paths=np.array(all_paths)
all_labels=np.array(all_labels)
train_lst,val_lst,train_labels,val_labels=train_test_split(all_paths,all_labels,test_size=0.3)
val_lst,test_lst,val_labels,test_labels=train_test_split(val_lst,val_labels,test_size=0.33)

In [ ]:
class DataSet(Dataset):
    def __init__(self,paths,labels,transform=None):
        self.transform=transform
        self.images_path=paths
        self.labels=labels
        if(transform is not None):
            self.images_path=np.append(self.images_path,self.images_path)
            self.labels=np.append(self.labels,self.labels)
    def __len__(self):
        return len(self.images_path)
    def __getitem__(self,idx):
        hf=len(self.images_path)/2
        img=self.images_path[idx]
        image=cv2.imread(img)
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        resized_image = cv2.resize(image,(299,299), interpolation=cv2.INTER_CUBIC)
        resized_image=np.array(resized_image)
        resized_image=torch.tensor(resized_image)
        if(self.transform is not None):
            half=len(self.images_path)/2
            if(idx>=half):
                resized_image=torch.reshape(resized_image,(3,299,299))
                resized_image=resized_image.to(torch.float32)
                resized_image=self.transform(resized_image)
                return resized_image,self.labels[idx]
        resized_image=torch.reshape(resized_image,(3,299,299))
        resized_image=resized_image.to(torch.float32)
        return resized_image,self.labels[idx]

In [ ]:
train_dataset=DataSet(train_lst,train_labels,trans)
val_dataset=DataSet(val_lst,val_labels,trans)
test_dataset=DataSet(test_lst,test_labels,trans)
train_dataloader=DataLoader(train_dataset,batch_size=16,shuffle=True)
val_dataloader=DataLoader(val_dataset,batch_size=16,shuffle=True)
test_dataloader=DataLoader(test_dataset,batch_size=16,shuffle=True)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,f1_score
def get_acc(pred,out):
    arr=pred.to('cpu').detach()
    arr=np.array(arr)
    arr2=out.to('cpu').detach()
    arr2=np.array(arr2)
    return accuracy_score(arr,arr2)
def get_prec(pred,out):
    arr=pred.to('cpu').detach()
    arr=np.array(arr)
    arr2=out.to('cpu').detach()
    arr2=np.array(arr2)
    return precision_score(arr,arr2,average='macro')
def get_f1(pred,out):
    arr=pred.to('cpu').detach()
    arr=np.array(arr)
    arr2=out.to('cpu').detach()
    arr2=np.array(arr2)
    return f1_score(arr,arr2,average='macro')

In [ ]:
def test(model,test_dataloader):
    model.eval()
    with torch.no_grad():
        total_loss = 0
        test_acc=0
        test_prec=0
        test_f1=0
        for i,(inputs, targets) in enumerate(test_dataloader):
            outputs = model(inputs)
            targets=targets.to(torch.long)
            targets=targets.to('cpu')
            outputs=outputs.to('cpu')
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            outputs=torch.argmax(outputs,dim=1)
            test_acc+=get_acc(outputs,targets)
            test_prec+=get_prec(outputs,targets)
            test_f1+=get_f1(outputs,targets)
        test_acc/=(i+1)
        total_loss/=(i+1)
        test_prec/=(i+1)
        test_f1/=(i+1)
        sen = f"Testing ,Running Loss {total_loss} ,Accuracy:{test_acc},Precision:{test_prec},F1:{test_f1}"
        print(sen)
        return total_loss,test_acc,test_prec,test_f1

In [ ]:
import pickle
def save_model(list_models,filename):
    pickle.dump(list_models, open(filename, 'wb'))
def load_model(filename):
    list_models=pickle.load(open(filename, 'rb'))
    return list_models
filename="Weights/Inception"
model=load_model(filename)

In [ ]:
total_loss,test_acc,test_prec,test_f1=test(model,test_dataloader)